In [52]:
import pandas as pd, openpyxl
import mysql.connector.connection
import os, sys, json
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
# Configuracion ruta json
with open("rutas.json", "r") as archivo:
    rutas = json.load(archivo)
    
def escribir_registro(texto):
    with open("registro.txt", "a",encoding="utf-8") as registro:
        rutas = registro.write(f"{texto}\n")
escribir_registro("\n\n")

Configuracion Principal

In [64]:
month = 3
year = 2025
codigo_empresa = 1521
tipo_pago = "M"

In [54]:
date = f"{month}-{year}"
date = datetime.strptime(date, "%m-%Y")
month_str = datetime.strftime(date, "%B")
file_name = f"{month_str} {year}.xlsx"
file_name

'March 2025.xlsx'

In [55]:
# Seccion lector inputs
df = pd.read_excel(f"{rutas['input']}Novedades {file_name}")
df.head()

,Legajo,Documento,Apellido,Nombre,Basico,D. Enfermedad,Vacaciones,Feriado,Licencia,D. Mudanza,...,Falta Inj.,Hs Extras,Dep. Judicial,Prepaga,Enlazados,Ant. de Sueldos,Adic. Ventas,OBSERVACIONES,Fecha baja,Fecha alta
0,1365,44059638,ABARCA,NOELIA AGUSTINA,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,1964,38417005,ABBONA,MATIAS ANDRES,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,1905,41339262,ABDO,LUCIA,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-06,NaT
3,1906,42976152,ACUÑA,IGNACIO ESTEBAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,2078,44746629,AGUILERA,ANDREA MACARENA,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2025-03-20


PROCESAMIENTO DE DATOS

El fin de este procesamiento es generar un dataframe en el cual me haga aparezca por asesor la cantidad de licencias o excepciones que tiene, compararlo con el codificado.



In [56]:
# Seccion limpieza de dataframe
df = df[["Legajo", "Basico", "Vacaciones", "Feriado", "D. Mudanza", "D. Estudio", "D. Suspension", "D. Tramite","Hs Extras", "Dep. Judicial", "Prepaga", "Enlazados", "Ant. de Sueldos", "Adic. Ventas"]]
df.head()

,Legajo,Basico,Vacaciones,Feriado,D. Mudanza,D. Estudio,D. Suspension,D. Tramite,Hs Extras,Dep. Judicial,Prepaga,Enlazados,Ant. de Sueldos,Adic. Ventas
0,1365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1905,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2078,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extraccion de informacion de la tabla payroll

Extraemos los datos de payoll para la identificacion de licencias y otros dias de ausencia. De este reporte solamente se sacara la informacion desde la columna d. enfermedad hasta la columna Hs Extras

In [57]:
def conexion_db(host, user, password, database):
    try:
        cnx = mysql.connector.connect(host= host, user=user, password=password, database=database)
        print("Conexion exitosa a base de datos")
        now = datetime.now()
        escribir_registro(f"[{now}] ✅ Conexion exitosa a base de datos")
        return cnx
    except Exception as conection_error:
        print(f"Se produjo un error al intentar conectarse a base de datos: \n{conection_error}")
        now = datetime.now()
        escribir_registro(f"[{now}] ❌ Ocurrio un error al conectarse a base de datos")
def extraccion_de_datos (cnx, query, tabla):
    try:
        cursor = cnx.cursor()
        cursor.execute(query)
        resultado = cursor.fetchall()
        if len(resultado) == 0:
            print("Advertencia. La consulta no trajo informacion") 
        cursor.close()
        cnx.close()
        now = datetime.now()
        escribir_registro(f"[{now}] ✅ Se extrajo correctamente la informacion a base de datos")
        print("Se extrajo la informacion correctamente de base de datos")
        return resultado
    except Exception as extract_error:
        now = datetime.now()
        print(f"Se produjo un error al extraer la informacion de base de datos {extract_error}\n {query}")
        escribir_registro(f"[{now}]❌ Ocurrio un error al extraer los datos de la tabla {tabla}")

In [58]:
# Consulta de extraccion payroll
query= f"""
SELECT 
legajo, 
fecha, 
codigo, 
descripcion,
horas_programadas, 
horas_trabajadas
FROM payroll  WHERE MONTH(fecha) = {month} AND YEAR(fecha) = {year}"""
columnas = ["legajo", "fecha", "codigo", "descripcion", "horas_programadas", "horas_trabajadas"]

# Consulta extraccion asesores
query_asesores = f"""
SELECT 
legajo,
apellidos, 
nombres
FROM asesores
WHERE fecha_baja is null or (month(fecha_baja) = {month} and year(fecha_baja) = {year})"""

In [59]:
host = os.getenv('DB_HOST')
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
tabla = "payroll"
cnx = conexion_db(host, user, password, "db_omnia")
resultado = extraccion_de_datos(cnx, query, tabla)
df_payroll = pd.DataFrame(resultado, columns=columnas)
df_payroll.head()
 

Conexion exitosa a base de datos
Se extrajo la informacion correctamente de base de datos


,legajo,fecha,codigo,descripcion,horas_programadas,horas_trabajadas
0,1098,2025-03-03,DNL,Día No Laborable,8.0,0.0
1,1098,2025-03-04,DNL,Día No Laborable,8.0,0.0
2,1098,2025-03-05,V,Vacaciones,0.0,0.0
3,1098,2025-03-06,V,Vacaciones,0.0,0.0
4,1098,2025-03-07,V,Vacaciones,0.0,0.0


In [60]:
codificado = {
    "LM": 409,  # LICENCIA POR MATERNIDAD
    "LFF": 56,  # FALLECIMIENTO
    "MTM": 55,  # MATRIMONIO
    "CE": 62,  # ESTUDIO
    "ART" : 65, # ENFERMEDAD
    "LMED": 65,  # ENFERMEDAD
    "LESP" : 65, # Enfermedad
    "CM" : 65, # Enfermedad
    "LPS" : 65, # Enfermedad - Licencia psiquiatrica
    "Prepaga" : 654,
    "Basico" : 1,
    "Vacaciones" : 17,
    "Feriado" : 8,
    "D. Mudanza" : 54,
    "D. Estudio" : 62,
    "D. Suspension" : 50,
    "D. Tramite" : 51,
    "Enlazados" : 675 
}


In [61]:
# Hay que realizar un conteo de codigos cuando estos sean los que queremos
""" Se puede realizar tambien una comparacion mediante una cadena de valores """
# iteramos por filas
try:
    info_recolectada_payroll = []
    for index, row in df_payroll.iterrows():
        if row['codigo'] in codificado:
            codigo = row['codigo'] 
            info_recolectada_payroll.append({
                "Empresa" : codigo_empresa,
                "Legajo" : row['legajo'],
                "Periodo" : year,
                "Mes" : month,
                "Concepto" : codificado[codigo],
                "Tipo" : tipo_pago,
                "Forzado" : 0,
                "Cantidad" : 1
            })
    df_borrar = pd.DataFrame(info_recolectada_payroll)
    df_borrar = df_borrar.groupby(by=['Empresa', 'Legajo','Periodo', 'Mes', 'Concepto', 'Tipo', 'Forzado'])["Cantidad"].sum().reset_index()
    df_borrar.to_excel(f"{rutas['Output']}procesado database.xlsx", index=False)
    now = datetime.now()
    escribir_registro(f"[{now}] ✅ Se codifico correctamente el dataset extraido de payroll")
    print(df_borrar)
except Exception as error:
    now = datetime.now()
    escribir_registro(f"[{now}]❌ Ocurrio un error en el modulo codificacionDatasetPayroll")
    print(f"❌ Ocurrio un error en el modulo", error)


     Empresa  Legajo  Periodo  Mes  Concepto Tipo  Forzado  Cantidad
0       1521     559     2025    3        65    M        0         4
1       1521     560     2025    3        65    M        0         1
2       1521     617     2025    3        62    M        0         2
3       1521     894     2025    3        65    M        0         3
4       1521     943     2025    3        65    M        0         1
..       ...     ...      ...  ...       ...  ...      ...       ...
184     1521    2035     2025    3        65    M        0        15
185     1521    2047     2025    3        65    M        0         3
186     1521    2065     2025    3        62    M        0         1
187     1521    2070     2025    3        65    M        0         1
188     1521    2075     2025    3        62    M        0         2

[189 rows x 8 columns]


In [62]:
resultado = []
errores = []
for i, fila in df.iterrows():
    for col in df.columns[1:]:
        try:  # Salteamos la primera columna (ID)
            if pd.notna(fila[col]):
                resultado.append({
                    "Empresa" : codigo_empresa,
                    "Legajo" : fila['Legajo'],
                    "Periodo" : year,
                    "Mes" : month,
                    "Concepto" : codificado[col],
                    "Tipo" : tipo_pago,
                    "Forzado" : 0,
                    "Cantidad" : fila[col]
                })
        except Exception as error_codificacion:
            print(f"Ha ocurrido un error en la codificacion del reporte extraido del excel {error_codificacion}")
            errores.append({
                "Legajo" : fila['Legajo'],
                "Codigo" : col
            })
            
# Control de errores
if len(errores) > 0:
    print("No se han cargado los siguientes asesores\n\n",errores)
    now = datetime.now()
    escribir_registro(f"[{now}]❌ Ocurrio un error al procesar el modulo 'codificacionNovedades'")
else:
    now = datetime.now()
    escribir_registro(f"[{now}] ✅ Se codifico correctamente el dataset extraido de la planilla de novedades")
            
df_novedades = pd.DataFrame(resultado)
if len(df_novedades) > 0:
    df_novedades['Legajo'] = df_novedades['Legajo'].astype(int)
    df_novedades['Cantidad'] = df_novedades['Cantidad'].astype(int)
    
df_novedades.head()

,Empresa,Legajo,Periodo,Mes,Concepto,Tipo,Forzado,Cantidad
0,1521,1905,2025,3,1,M,0,6
1,1521,2078,2025,3,1,M,0,10
2,1521,1187,2025,3,1,M,0,18
3,1521,1770,2025,3,62,M,0,2
4,1521,1426,2025,3,1,M,0,24


In [63]:
df_final = pd.concat([df_borrar, df_novedades],axis=0)
df_final = df_final.groupby(by=['Empresa', 'Legajo','Periodo', 'Mes', 'Concepto', 'Tipo', 'Forzado'])['Cantidad'].sum().reset_index()

file_name = file_name.replace(".xlsx", "")
df_final.to_excel(f"{rutas['Output']}E-Sueldos {file_name}.xlsx", index= False)
now = datetime.now()
escribir_registro(f"> Se obtuvieron {len(df_novedades)} filas de novedades")
escribir_registro(f"> Se obtuvieron {len(df_borrar)} filas de payroll")
escribir_registro(f"> Se exportaron {len(df_final)} filas")
